In [ ]:
import urllib
import zipfile
import geopandas as gpd
import pandas as pd

url_download = r'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_16_2020_csv.zip'
input_path = 'inputs/'
urllib.request.urlretrieve(url_download, 'RESAGEBURB_16_2020_csv.zip')
unzip = zipfile.ZipFile('RESAGEBURB_16_2020_csv.zip')
unzip.extractall(input_path)

url_download = r'https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463849568/16_michoacandeocampo.zip'
input_path = 'inputs/'
urllib.request.urlretrieve(url_download, '16_michoacandeocampo.zip')
unzip = zipfile.ZipFile('16_michoacandeocampo.zip')
unzip.extractall(input_path)

In [ ]:
michoacan_full = pd.read_csv('inputs/RESAGEBURB_16CSV20.csv')

In [ ]:
michoacan_shp = gpd.read_file('inputs/conjunto_de_datos/16m.shp')

In [ ]:
morelia = michoacan_full.loc[(michoacan_full["MUN"] ==53) & (michoacan_full["MZA"] !=0) ]

In [ ]:
morelia.to_csv('outputs/morelia_manzanas_original.csv')

In [ ]:
morelia["ENTIDAD"] = morelia["ENTIDAD"].astype(str)

In [ ]:
morelia['MUN'] = morelia['MUN'].apply(lambda x: '{0:0>3}'.format(x))
morelia['LOC'] = morelia['LOC'].apply(lambda x: '{0:0>4}'.format(x))
morelia['MZA'] = morelia['MZA'].apply(lambda x: '{0:0>3}'.format(x))
morelia['CVEGEO']= morelia['ENTIDAD'] + morelia['MUN'] +morelia['LOC'] +morelia['AGEB'] +morelia['MZA']


In [ ]:
morelia_shp = michoacan_shp.loc[(michoacan_shp["CVE_MUN"] =='053') ]

In [ ]:
morelia_shp.to_file('outputs/morelia_original.shp')

In [ ]:
shape_full = pd.merge(morelia_shp, morelia[['MUN','LOC','MZA','CVEGEO','POBTOT','P_18YMAS','TVIVHAB']], on = 'CVEGEO')

In [ ]:
shape_full.to_file('outputs/morelia_con_poblacion.shp')

In [ ]:
morelia_urbano = shape_full.loc[(shape_full["AMBITO"] =='Urbana') ]


In [ ]:
morelia_urbano = morelia_urbano.replace({'*': '0'})

In [ ]:
morelia_urbano_habitado = morelia_urbano.loc[(morelia_urbano["TVIVHAB"] !=0) ]

In [ ]:
morelia_urbano_habitado.to_file('outputs/morelia_urbano_habitado.shp')
morelia_urbano_habitado.to_csv('outputs/morelia_urbano_habitado.csv')

In [ ]:
colonias_michoacan = gpd.read_file("inputs/COLONIA.shp" )

In [ ]:
colonias_morelia = colonias_michoacan.loc[(colonias_michoacan["MUNICIPIO"] ==54) ]


In [ ]:
colonias_morelia = colonias_morelia.to_crs('EPSG:4326')


In [ ]:
morelia_urbano_habitado = morelia_urbano_habitado.to_crs('EPSG:4326')

In [ ]:
morelia_urbano_habitado['centroid'] = morelia_urbano_habitado.centroid
join = gpd.sjoin(colonias_morelia, morelia_urbano_habitado.set_geometry('centroid'), how="right", predicate="contains")


In [ ]:
join.P_18YMAS = join.P_18YMAS.fillna(0)

In [ ]:
join.TVIVHAB = join.TVIVHAB.fillna(0)

In [ ]:
join.P_18YMAS = join.P_18YMAS.astype('int64')

In [ ]:
join.TVIVHAB = join.TVIVHAB.astype('int64')

In [ ]:
habitantes_colonia = join.groupby(['NOMBRE'])[['POBTOT', "P_18YMAS", "TVIVHAB"]].agg('sum').reset_index()

In [ ]:
join.to_csv('outputs/paso_faltante.csv')

In [ ]:
pob_col_mor = pd.merge(colonias_morelia, habitantes_colonia,on='NOMBRE')

In [ ]:
pob_col_mor.to_file('outputs/poblacion_colonias_morelia.shp')
pob_col_mor.to_csv('outputs/poblacion_colonias_morelia.csv')

In [ ]:
habitantes_colonia

In [ ]:

cuadrantes_morelia = gpd.read_file("inputs/redudasproyectomorelia (1)/sectores_processed.shp" )




In [ ]:
cuadrantes_morelia = cuadrantes_morelia.to_crs('EPSG:4326')

In [ ]:
pob_col_mor['centroid'] = pob_col_mor.centroid

In [ ]:

join_cuadrantes = gpd.sjoin(cuadrantes_morelia, pob_col_mor.set_geometry('centroid'), how="left", predicate="contains")

In [ ]:

join_cuadrantes.to_csv('outputs/colonias_con_poblacion_y_cuadrantes.csv')


In [ ]:
join_cuadrantes

In [ ]:

tabla_cuadrantes = join_cuadrantes.groupby('Sector')['POBTOT',"P_18YMAS",'TVIVHAB'].sum()

In [ ]:
tabla_cuadrantes.to_csv('outputs/tabla_poblacion.csv')